In [1]:
%pip install -q trino

Note: you may need to restart the kernel to use updated packages.


In [1]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication
from redirect_handler import REDIRECT_HANDLER
import urllib3
import pandas as pd

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

TRINO_URI = "https://trino-proxy:443"

# Use the Catalog (User 1: Peter)
Now that the catalog is created, we can use it as usual. Lets login again (if asked) as `peter`, this time directly to the catalog so that we don't have to prefix schemas with `lakekeeper`:

In [17]:
conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
    catalog="lakekeeper" # This line is new
)

In [18]:
cur = conn.cursor()
cur.execute("CREATE SCHEMA IF NOT EXISTS finance")

Open the following URL in browser for the external authentication:
http://localhost:38191/oauth2/token/initiate/a95f15c7f64b465a9d8e3fc661c56df008ae4677ef88c866abe46b424509e1fe


TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot access catalog lakekeeper", query_id=20251003_032311_00019_zuqhg)

In [16]:
cur.execute("CREATE TABLE IF NOT EXISTS finance.revenue (year INT, revenue DOUBLE)")
cur.execute("INSERT INTO finance.revenue VALUES (2026, 10355.1), (2025, 10477.2)")


TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot execute query", query_id=20251003_032255_00018_zuqhg)

In [7]:
# Execute query and fetch all rows
cur.execute("SELECT * FROM finance.revenue")
rows = cur.fetchall()

# Get column names
columns = [desc[0] for desc in cur.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display DataFrame
print(df)

   year  revenue
0  2025  10333.1
1  2023  10342.1
2  2024  10645.2
3  2024  10444.2


In [8]:
cur.execute("CREATE TABLE IF NOT EXISTS finance.products (product_id INT, description VARCHAR)")
cur.execute("INSERT INTO finance.products VALUES (3, 'Product 3'), (4, 'Product 4')")


In [9]:
# Execute query and fetch all rows
cur.execute("SELECT * FROM finance.products")
rows = cur.fetchall()

# Get column names
columns = [desc[0] for desc in cur.description]

# Create DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display DataFrame
print(df)

   product_id description
0           1   Product 1
1           3   Product 3
2           4   Product 4
3           2   Product 2
